In [36]:
import os
import torch
import random
import numpy as np
import json
import torch.nn as nn

In [37]:
def loadmodel(name):
    x = open('model_parameters/'+ name+'.json')
    data = json.load(x)
    model_structure = data['model_structure']
    model_parameters_names = data['model_parameters_names']
    model_parameters_vals_ = data['model_parameters_vals']
    model_parameters_vals=[]
    for i in model_parameters_vals:
        model_parameters_vals.append(list(map(lambda x: x + random.randint(-10,10)*0.000001, i)))
    model_parameters_names = [f'[{name.split(".")[0]}].{name.split(".")[1]}'
                                   for name in model_parameters_names.split('|')]
    received_model = nn.Sequential(
    *[eval('nn.' + layer) for layer in model_structure.split('|')]
    )
    for parameters_name, parameters_val in zip(model_parameters_names, model_parameters_vals):
        exec(f'received_model{parameters_name} = nn.Parameter(torch.tensor(parameters_val))')
    for param in received_model.parameters():
        param.requires_grad = False
    print()
    return received_model.to('cuda')

In [ ]:
torch.load('saved_models/model_A')


In [38]:
def loadmodels(names):
    all=[]
    for i in names:
        all.append(loadmodel(i))
    return all[0]


In [39]:
from predict import predict_from_dataset

In [40]:
dataset_path = '../../Datasets/CIFAR10'

In [41]:
def ensemble(models,path,device):
    for i in models:
        predict_from_dataset(path,'A',"sb","sb",models,device)

In [42]:
def final(path):
    models= loadmodels(["model_A"])
    y= ensemble(models,dataset_path,'cuda')
    return y

In [ ]:
 final(dataset_path)

<torch.utils.data.dataloader.DataLoader object at 0x000001BF67A5A850> Sequential(
  (0): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): Linear(in_features=768, out_features=128, bias=True)
  (2): GELU()
  (3): Linear(in_features=128, out_features=6, bias=True)
)


 60%|██████    | 255/422 [04:59<03:15,  1.17s/it]